In [4]:
import urllib.request
 
from multiprocessing.dummy import Pool as ThreadPool
 




import requests
from bs4 import BeautifulSoup
import time


import time
import datetime
from functools import wraps


def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = datetime.datetime.now()
        print(t0)
        result = function(*args, **kwargs)
        t1 = datetime.datetime.now()
        print(t1)
        print("Total time running %s: %s seconds" %
              (function.__name__, str(t1 - t0))
              )
        return result

    return function_timer
def fn_timer_microsecond(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = datetime.datetime.now().microsecond
        print(datetime.datetime.now())
        result = function(*args, **kwargs)
        t1 = datetime.datetime.now().microsecond
        print(datetime.datetime.now())
        print("Total time running %s: %s seconds" %
              (function.__name__, str(t1 - t0))
              )
        return result

    return function_timer

URL = 'http://sh.lianjia.com'
xiaoqu_url = 'http://sh.lianjia.com/xiaoqu/d'
detail_url = 'http://sh.lianjia.com/xiaoqu/5011102207057.html'



def download_page(url, retries=3):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
    try:
        data = requests.get(url, headers=headers).content
    except Exception as err:
        print("fail to dowload the page, reason: ")
        print(err)
        if retries>0:
            print("try again")
            return download_page(url,retries-1)
        else:
            print("failed in scaping : %s" % url)
            return ''        
    
    return data


def parse_page(html):
    soup = BeautifulSoup(html,"html.parser")
    link_list = []
    name_list = []
    xiaoqu_list = soup.findAll('div', attrs={'class': 'info-panel'})
    # print(xiaoqu_list)

    for xiaoqu in xiaoqu_list:
        link = xiaoqu.find('a').get('href')
        name = xiaoqu.find('a').get('title')
        link_list.append(link)
    
    for link in link_list:
        
        parse_detail_page(link)


def parse_detail_page(link):
    result = ' '
    detail_link = URL + link
    detail_soup = BeautifulSoup(download_page(detail_link),"html.parser")
    # get the price
    price = detail_soup.find('span',attrs={'class':'p'}).getText().strip()

    # get name, latitude and longitude
    coord = detail_soup.find('a',attrs={'class':'actshowMap'}).get('xiaoqu')

    coord = coord.split(',')

    latitude = coord[1]
    longitude = coord[0].strip('[')
    # name = coord[2].strip(']').strip("'")
    name = coord[2][2:-2]

    # get address
    adr = detail_soup.find('span',attrs={'class':'adr'}).getText().strip()

    result = name + ' ' + link + ' ' + price + ' ' + latitude + ' ' + longitude + ' ' + adr

    # get other info
    others = []
    other_info = detail_soup.findAll('span',attrs={'class':'other'})
    # print(other_info)
    for info in other_info:
        # print(info.getText())
        # print(info.getText().strip())
        others.append(info.getText().strip())
    
    
    i = 1
    for other in others:
        if i<3:
            result += ' ' + other
            i += 1
        else:
            break
  
    result +=  '\n'
    print(result)
    with open('shanghai_old.txt', 'a') as f:
        f.write(result)
    
    # time.sleep(0.5)

def parse_detail_page_test(html):

    detail_soup = BeautifulSoup(html,"html.parser")
    # get the price
    price = detail_soup.find('span',attrs={'class':'p'}).getText().strip()

    # get name, latitude and longitude
    coord = detail_soup.find('a',attrs={'class':'actshowMap'}).get('xiaoqu')

    coord = coord.split(',')

    latitude = coord[1]
    longitude = coord[0].strip('[')
    # name = coord[2].strip(']').strip("'")
    name = coord[2][2:-2]

    # get address
    adr = detail_soup.find('span',attrs={'class':'adr'}).getText().strip()

#     result = name + ' ' + link + ' ' + price + ' ' + latitude + ' ' + longitude + ' ' + adr
    result = name + ' ' + price + ' ' + latitude + ' ' + longitude + ' ' + adr


    # get other info
    others = []
    other_info = detail_soup.findAll('span',attrs={'class':'other'})
    # print(other_info)
    for info in other_info:
        # print(info.getText())
        # print(info.getText().strip())
        others.append(info.getText().strip())
    
    
    i = 1
    for other in others:
        if i<3:
            result += ' ' + other
            i += 1
        else:
            break
  
    result +=  '\n'
    print(result)
    with open('shanghai_new.txt', 'a') as f:
        f.write(result)
    
    # time.sleep(0.5)
    
    
def parse_page_test(html):
    soup = BeautifulSoup(html,"html.parser")
    link_list = []
    name_list = []
    xiaoqu_list = soup.findAll('div', attrs={'class': 'info-panel'})
    # print(xiaoqu_list)

    for xiaoqu in xiaoqu_list:
        link = xiaoqu.find('a').get('href')
        detail_link = URL + link
        link_list.append(detail_link)
    
    return link_list

@fn_timer
def m1():
    
    url = xiaoqu_url + '1'
    html = download_page(url)
    urls = parse_page_test(html)
     
    pool = ThreadPool()

#     results = pool.map(urllib.request.urlopen,urls)
    results = pool.map(download_page,urls)
#     print(results)
    for http_obj in results:
        parse_detail_page_test(http_obj)
    pool.close()
    pool.join()

    print('main1 ended')
    # detail_soup = BeautifulSoup(download_page(detail_url),"html.parser")
    # coord = detail_soup.find('a',attrs={'class':'actshowMap'}).get('xiaoqu')
    
    # parse_detail_page('hh', detail_url)
    # a = "'绿地威廉公寓']"
    # print(a)
    # print(a[1:-2])
    # print(a.strip("]").strip("'"))
    # a = a.strip("'")
    # print(a)

#     url = xiaoqu_url + '1'
#     html = download_page(url)
#     parse_page(html)

 
#     for i in range(89,101):
        
#         url = xiaoqu_url + str(i)
#         print("start scraping " + url)
#     # url = xiaoqu_url
#         html = download_page(url)
#         parse_page(html)
#         print("# ----------------- got page %d !!!" % i)

    # while url:
    #     html = download_page(url)
    #     movies, url = parse_page(html)
    #     print(movies)
        #print('--------------------------------------------------------------------------------------')
@fn_timer
def m2():
    
#     url = xiaoqu_url + '1'
#     html = download_page(url)
#     urls = parse_page_test(html)
     
#     pool = ThreadPool()

# #     results = pool.map(urllib.request.urlopen,urls)
#     results = pool.map(download_page,urls)
# #     print(results)
#     for http_obj in results:
#         parse_detail_page_test(http_obj)
#     pool.close()
#     pool.join()

#     print('main ended')
    # detail_soup = BeautifulSoup(download_page(detail_url),"html.parser")
    # coord = detail_soup.find('a',attrs={'class':'actshowMap'}).get('xiaoqu')
    
    # parse_detail_page('hh', detail_url)
    # a = "'绿地威廉公寓']"
    # print(a)
    # print(a[1:-2])
    # print(a.strip("]").strip("'"))
    # a = a.strip("'")
    # print(a)

    url = xiaoqu_url + '1'
    html = download_page(url)
    parse_page(html)
    print('main2 ended')

#     for i in range(89,101):       
#         url = xiaoqu_url + str(i)
#         print("start scraping " + url)
#     # url = xiaoqu_url
#         html = download_page(url)
#         parse_page(html)
#         print("# ----------------- got page %d !!!" % i)

    # while url:
    #     html = download_page(url)
    #     movies, url = parse_page(html)
    #     print(movies)
        #print('--------------------------------------------------------------------------------------')
def main():
    m1()
    m2()
if __name__ == '__main__':
    main()
    print('good')
# -------------------------------------------------------------------------------------------------------





2016-12-01 15:32:56.796746
fail to dowload the page, reason: 
HTTPConnectionPool(host='sh.lianjia.com', port=80): Max retries exceeded with url: /xiaoqu/5011000017980.html (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000024B77F48E80>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))
try again
上海康城 40508  31.10631595 121.362808 莘松路958弄（大浪湾道01-61号、山林道04-96号、江山道01-23号、瀑布湾道21-96号、维园道01-45号、康城道51-82） 住宅 - 公寓 2008年建

中远两湾城 73305  31.259226 121.451405 中潭路33弄,远景路97弄,中潭路99弄,中潭路100弄,中潭路91弄 住宅 - 公寓 2006年建

奥林匹克花园 48850  31.15272248 121.3362569 涞亭北路99弄,涞寅路106弄 住宅 - 公寓 2010年建

新城金郡 36539  31.34873307 121.2462989 临泽路88弄, 临泽路89弄, 临泽路188弄, 临泽路189弄, 云屏路1399弄, 崇信路1518弄, 白银路1565号, 白银路1563号 住宅 - 公寓、商住、住宅 - 写字楼 2013年建

绿洲康城亲水湾 56919  31.158247 121.590724 康桥路1259弄, 康桥路1558弄, 康桥路1633弄, 康桥路1657弄 住宅 - 公寓 2017年建

保利叶上海（公寓） 57726  31.35350639 121.3651601 联杨路1078弄，菊联路262弄，菊太路1198弄 住宅 - 公寓 2012年建

证大家园（公寓） 77255  31.2